In [1]:
import json, time
import pandas as pd
import gzip
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from bs4 import BeautifulSoup

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Clothing_Shoes_and_Jewelry_5.json.gz')

In [3]:
#Create a pivot table with the product ID as the rows, the userID as the columns and the overall rating as the values.
pivot = df.pivot_table(index='asin',
              columns='reviewerID',
              values='overall')

In [4]:
pivot.head()

reviewerID,A001114613O3F18Q5NVR6,A00146182PNM90WNNAZ5Q,A00165422B2GAUE3EL6Z0,A00338282E99B8OR2JYTZ,A00354001GE099Q1FL0TU,A00404823GU1Q517KP4Z8,A00475741WVLQL5CVWK2Z,A0055084JB2YQW2IDOSQ,A00635603LUUJQPQWSJW1,A007227915NZZC7B6P030,...,AZZ2HXC308RW1,AZZ9BV67BM0V0,AZZHJIL9S6XZ5,AZZHZZMH3U1VB,AZZKXNSXR6KBE,AZZMQ85DPFEG3,AZZNK89PXD006,AZZT1ERHBSNQ8,AZZTOUKVTUMVM,AZZYW4YOE1B6E
asin,,,,,,,,,,,,,,,,,,,,,
0000031887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0123456479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608299953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1617160377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00001W0KA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Shape matches the num of unique users in the columns and num of unique items in the rows 
pivot.shape

(23033, 39387)

In [6]:
#Convert to sparse matrix to make smaller file size 
pivot_sparse = sparse.csr_matrix(pivot.fillna(0))

In [7]:
print(pivot_sparse[:5])

  (0, 1824)	5.0
  (0, 5623)	5.0
  (0, 5862)	5.0
  (0, 6526)	5.0
  (0, 6712)	4.0
  (0, 7923)	5.0
  (0, 12214)	3.0
  (0, 12672)	4.0
  (0, 12790)	5.0
  (0, 13319)	5.0
  (0, 15118)	5.0
  (0, 16827)	5.0
  (0, 18008)	5.0
  (0, 18835)	5.0
  (0, 20187)	1.0
  (0, 22176)	5.0
  (0, 23539)	5.0
  (0, 25109)	5.0
  (0, 25705)	5.0
  (0, 31651)	5.0
  (0, 34621)	5.0
  (0, 34843)	4.0
  (0, 38603)	5.0
  (1, 5496)	3.0
  (1, 9345)	5.0
  :	:
  (2, 36879)	4.0
  (3, 2497)	4.0
  (3, 10528)	5.0
  (3, 10727)	5.0
  (3, 11951)	5.0
  (3, 13803)	5.0
  (3, 16042)	4.0
  (3, 19585)	4.0
  (3, 21346)	5.0
  (3, 22995)	5.0
  (3, 26140)	4.0
  (3, 27419)	5.0
  (3, 29666)	4.0
  (3, 35757)	4.0
  (3, 36104)	5.0
  (3, 38888)	5.0
  (3, 39186)	5.0
  (4, 112)	4.0
  (4, 601)	5.0
  (4, 4484)	4.0
  (4, 11432)	5.0
  (4, 19590)	4.0
  (4, 19631)	5.0
  (4, 23062)	5.0
  (4, 31246)	5.0


In [8]:
# Check similiarity of every item to every other item. 
recommender = pairwise_distances(pivot_sparse, metric='cosine')

In [9]:
#Checking to make sure it's a square. 
recommender.shape

(23033, 23033)

In [10]:
#Convert to DataFrame 
recommender_df = pd.DataFrame(recommender, index=pivot.index, columns=pivot.index)

In [11]:
recommender_df.head()

asin,0000031887,0123456479,1608299953,1617160377,B00001W0KA,B00001WRHJ,B00004SR8W,B00004SR8Z,B00004SR9P,B00004U1J2,...,B00K0BPFY0,B00K551QR6,B00K5T4NHC,B00K8J06CK,B00KA2X4QK,B00KA602SY,B00KCWMG5S,B00KF9180W,B00KGCLROK,B00KKXCJQU
asin,,,,,,,,,,,,,,,,,,,,,
0000031887,0.0,1.0,1.0000,1.0000,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0
0123456479,1.0,0.0,1.0000,1.0000,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0
1608299953,1.0,1.0,0.0000,0.9328,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0
1617160377,1.0,1.0,0.9328,0.0000,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,0.895685,1.0,1.0,1.0,1.0,0.911762,0.868209,1.0
B00001W0KA,1.0,1.0,1.0000,1.0000,0.0,1.0,1.0,1.0,1.0,0.574821,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0


In [12]:
recommender_df['B00001W0KA'].sort_values()[1:11]

asin
B00004U1J2    0.574821
B005JJ2PK0    0.802905
B00551RXJK    0.810872
B006PHRPG4    0.810872
B0072QV4V4    0.814727
B0009ETG02    0.815289
B00EF0XQAW    0.826715
B004JV74VY    0.829995
B004M47EB8    0.847943
B00DNKFAT0    0.853357
Name: B00001W0KA, dtype: float64

In [ ]:
# A rough attempt at building a recommender and I see that it is important that I identify the product names 
# to match with the asin.  

In [13]:
recommender_df['0000031887'].sort_values()[1:11]

asin
B004QVDFWO    0.839901
B003IEDM9Q    0.856437
B004I72CMU    0.872767
B003EYVRUG    0.878977
B001F4SLAG    0.879691
B001HEB0R0    0.883360
B008FRFKBO    0.883725
B008NBSBHW    0.887954
B001C929N4    0.889080
B00B7TBDTU    0.892770
Name: 0000031887, dtype: float64

In [14]:
recommender_df['1608299953'].sort_values()[1:11]

asin
B000YM9P96    0.801562
B006H1CJU0    0.810519
B00280M8HS    0.839691
B0043G0JXK    0.848237
B003S9X7MI    0.848947
B000MQHHB2    0.848947
B004UULGQ8    0.848947
B006FTBRZC    0.848947
B001B7J4IA    0.848947
B001TI5H0A    0.850340
Name: 1608299953, dtype: float64

In [15]:
recommender_df['B004UULGQ8'].sort_values()[1:11]

asin
B000IPMFZK    0.766355
B005OIQ1BK    0.783831
B001TQ41VI    0.789641
B000W9Z8II    0.791217
B0074O2YKY    0.799292
B002L6XST6    0.801337
B005GNBY42    0.806653
B0099690ZM    0.813084
B005SITQJK    0.820852
B001941TTC    0.822789
Name: B004UULGQ8, dtype: float64